In [1]:
import feedparser
import hashlib
from threading import Timer
import time
from pandas import DataFrame
import datetime
import pandas as pd
import os.path
import csv

In [2]:
def write_dump(path, data):
    keys = data[0].keys()
    with open(path, 'wb') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

In [3]:
def open_dump(path):
    if os.path.isfile(path):
        res = []
        with open(path) as csvfile:
            reader = csv.DictReader(csvfile)
            for r in reader:
                res.append(r)
            return res    

In [4]:
dump_file = "news.csv"
set_columns = ['dt', 'ts', 'title', 'hash']

def scrap(hashes):
    ts = int(time.time())
    dt =  pd.to_datetime(datetime.datetime.now())
    data_temp = []
    feed_raw = feedparser.parse('https://fivethirtyeight.com/features/feed/')
    feed_entries = feed_raw['entries']

    for f in feed_entries:
        title = f['title'].encode('utf-8')
        hashed_title = hashlib.md5(title).hexdigest()

        if not hashed_title in hashes:
            dt = dt + datetime.timedelta(minutes=30) ## just for test !!!!
            entry = {'dt': dt, 'ts' : ts, 'title' :  title, 'hash': hashed_title} 
            data_temp.append(entry)

    #df = DataFrame.from_records(data_temp, columns = set_columns) 
    return data_temp

def run_dump():
    data = open_dump(dump_file)
    if data is None or data == []:
        print "No CSV dump"
        # first start without csv
        res = scrap([])
        print res
        write_dump(dump_file, res)
        #res.to_csv(path_or_buf=dump_file, columns = set_columns)
    else:
        hashes = [e['hash'] for e in data]
        res = scrap(hashes)
        data = data + res
        write_dump(dump_file, data)
        #data.to_csv(path_or_buf=dump_file, columns = set_columns)
    Timer(10, run_dump).start()
 

In [5]:
run_dump()      

In [6]:
def read_dump(path):
    return pd.read_csv(path, index_col=0,parse_dates=[0])

In [7]:
def freq():
    df = read_dump(dump_file)
    #print pd.to_datetime(data['dt'], format='%Y-%m-%d %H:%M:%S.%f')
    group = data.groupby(pd.TimeGrouper(freq='1H'))

    group_counts = group.agg({'title':'count'})

    delta = group_counts.index.max() - group_counts.index.min()
    total = float(group_counts['title'].sum())
    total_hours =  delta.seconds * 60 * 60 
    frequency = total / total_hours
    return frequency



In [390]:
print "news pear hour: ", freq()

news pear hour:  1.7146776406e-07
